In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

from pymongo import MongoClient
from bson.json_util import dumps

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from AnimalShelter import ANIMAL_SHELTER

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 31567
db_name = "AAC"
collection_name = "animals"

# Connect to database via CRUD Module
db = ANIMAL_SHELTER(username, password, host, port, db_name, collection_name)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('CS-340 Dashboard - Denise Kendall'))),
    html.Hr(),
    html.Div(
        
        dcc.RadioItems(
            id='radio-item',
            labelStyle={'display':'inline-block'}, 
            options=[
                {'label':'Water Rescue', 'value': 'Water Rescue'}, 
                {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'}, 
                {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster Rescue or Individual Tracking'},
                {'label':'Reset', 'value': 'Reset'}], 
            value='Reset', 
            )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'height': '400 px', 'overflowY': 'auto', 'overflowX':'auto'},
        style_cell={'textAlign':'left'},
        style_header={
            'backgroundColor':'rgb(30, 30, 30)',
            'fontWeight': 'bold',
            'color': 'white'
        },
        style_data={
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(86, 86, 86)',
            }
        ],
                         
        #tooltips
        tooltip = {i: {
            'value': i,
            'use_with': 'both'
        } for i in df.columns},
        tooltip_delay=0,
        tooltip_duration=None,
        page_size=15,
        row_selectable="single",
        selected_rows=[0],
        selected_columns=[],
        column_selectable=False,
        row_deletable=False,
        editable=False,
        page_action="native",
        page_current=0,
    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('radio-item', 'value')])
def update_dashboard(radio_item):
        
        if radio_item == 'Water Rescue':
            records = db.read({
                '$and': [
                    {'$or': [ 
                        {'breed':'Labrador Retriever Mix'}, 
                        {'breed':'Chesapeake Bay Retriever'}, 
                        {'breed':'Newfoundland'}
                    ]},
                    {'sex_upon_outcome': 'Intact Female'}, 
                    { "$and": [
                            {'age_upon_outcome_in_weeks':{"$gt": 26}},
                            {'age_upon_outcome_in_weeks':{"$lt": 156}}
                    ]}
                ]}
            )
        
        elif radio_item == 'Mountain or Wilderness Rescue':
            records = db.read({
                '$and': [
                    {'$or': [ 
                        {'breed':'German Shepherd'}, 
                        {'breed':'Alaskan Malamute'}, 
                        {'breed':'Old English Sheepdog'}, 
                        {'breed':'Siberian Husky'}, 
                        {'breed':'Rottweiler'}
                    ]},
                    {'sex_upon_outcome':'Intact Male'}, 
                    { "$and": [
                            {'age_upon_outcome_in_weeks':{"$gt": 26}},
                            {'age_upon_outcome_in_weeks':{"$lt": 156}}
                    ]}
                ]}
            )
            
        elif radio_item == 'Disaster Rescue or Individual Tracking':
            records = db.read({
                '$and': [
                    {'$or': [ 
                        {'breed': 'Doberman Pinscher'}, 
                        {'breed': 'German Sheperd'}, 
                        {'breed': 'Golden Retriever'}, 
                        {'breed':'Bloodhound'}, 
                        {'breed': 'Rottweiler'}
                    ]},
                    {'sex_upon_outcome':'Intact Male'}, 
                    { "$and": [
                            {'age_upon_outcome_in_weeks':{"$gt": 20}},
                            {'age_upon_outcome_in_weeks':{"$lt": 300}}
                    ]}
                ]}
            )
            
        else:
            records = db.read({})
        
        df = pd.DataFrame.from_records(records)
        df.drop(columns=['_id'],inplace=True)
        return df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData, **kwargs):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    
    df = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.pie(df, names=df['breed'], title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(port=3002)

Dash app running on http://127.0.0.1:3002/
